https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/image-search/Image_Search-multilingual.ipynb

# Multilingual Joint Image & Text Embeddings 

This example shows how [SentenceTransformers](https://www.sbert.net) can be used to map images and texts to the same vector space. 

As model, we use the [OpenAI CLIP Model](https://github.com/openai/CLIP), which was trained on a large set of images and image alt texts.

The original CLIP Model only works for English, hence, we used [Multilingual Knowlegde Distillation](https://arxiv.org/abs/2004.09813) to make this model work with 50+ languages.

As a source for fotos, we use the [Unsplash Dataset Lite](https://unsplash.com/data), which contains about 25k images. See the [License](https://unsplash.com/license) about the Unsplash images. 

Note: 25k images is rather small. If you search for really specific terms, the chance are high that no such photo exist in the collection.

In [1]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image, ImageFile
import glob
import torch
import pickle
import zipfile
from IPython.display import display
from IPython.display import Image as IPImage
import os
from tqdm import tqdm
import numpy as np

C:\Users\Miki\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
img_folder = 'images_30k/'

In [2]:
import pandas as pd
test_df = pd.read_csv(f"test_df_from_30k.csv")

In [3]:
test_df

,image,caption_number,caption,id
0,1001545525.jpg,0,Két férfi Németországban egyszerre ugrott át a...,9
1,1001545525.jpg,1,Két fiatal ugrál át az út menti korláton éjszaka.,9
2,1001545525.jpg,2,Rúdon táncoló fiúk az éjszaka közepén.,9
3,1001545525.jpg,3,Két ing nélküli férfi ugrál át a sínen.,9
4,1001545525.jpg,4,két srác együtt ugrott át egy kapun,9
...,...,...,...,...
31780,99804383.jpg,0,"Egy idősebb, szemüveges busker egy keleti vonó...",31781
31781,99804383.jpg,1,Ez egy férfi egy bolt előtt fellép egy kisfiúv...,31781
31782,99804383.jpg,2,Egy idősebb ázsiai férfi hangszeren játszik eg...,31781
31783,99804383.jpg,3,Egy idős férfi ül a kirakat előtt egy fiatal f...,31781


In [4]:
captions_w_fnames = test_df.to_dict('records')

In [5]:
import pprint
pp = pprint.PrettyPrinter(indent=4, width=200, depth=None, stream=None, compact=False, sort_dicts=False)

pp.pprint(captions_w_fnames[0].keys())
pp.pprint(captions_w_fnames[0])

dict_keys(['image', 'caption_number', 'caption', 'id'])
{'image': '1001545525.jpg', 'caption_number': 0, 'caption': 'Két férfi Németországban egyszerre ugrott át a sínen ing nélkül.', 'id': 9}


In [6]:
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

In [7]:
# we need a dict with an image name, and 5 captions
d_imgs_w_texts = {} 
for e in captions_w_fnames:
    append_value(d_imgs_w_texts, e['image'], e['caption'])

In [8]:
pp.pprint(d_imgs_w_texts)

{   '1001545525.jpg': [   'Két férfi Németországban egyszerre ugrott át a sínen ing nélkül.',
                          'Két fiatal ugrál át az út menti korláton éjszaka.',
                          'Rúdon táncoló fiúk az éjszaka közepén.',
                          'Két ing nélküli férfi ugrál át a sínen.',
                          'két srác együtt ugrott át egy kapun'],
    '1001633352.jpg': [   'Három fiatal férfi és egy tornacipőt viselő fiatal nő ugrál a levegőben egy betonlépcső tetején.',
                          'Négy lazán öltözött srác ugrál le egy lépcsőn a szabadban kőfallal a hátuk mögött.',
                          'Négy srác, három kalapban, egy nem ugrik a lépcső tetején.',
                          'Négy izgatott arcú férfi ugrik le a lépcső tetejéről.',
                          'Négy ember ugrál le egy lépcső tetejéről.'],
    '1001896054.jpg': [   'Egy férfi fényvisszaverő biztonsági ruhában és fülvédővel közlekedik egy John Deere traktorral az úton.',
          

In [9]:
d_imgs_w_texts.keys()

dict_keys(['1001545525.jpg', '1001633352.jpg', '1001896054.jpg', '1002674143.jpg', '100652400.jpg', '1007129816.jpg', '10101477.jpg', '1014609273.jpg', '1014785440.jpg', '10160966.jpg', '1018148011.jpg', '1021439420.jpg', '1022975728.jpg', '1024613706.jpg', '102617084.jpg', '10287332.jpg', '1030985833.jpg', '103306033.jpg', '1035019794.jpg', '10404007.jpg', '1040426962.jpg', '1044764620.jpg', '1045124251.jpg', '10459869.jpg', '1047921035.jpg', '104835889.jpg', '1051205546.jpg', '1051290485.jpg', '105223874.jpg', '105335409.jpg', '1053804096.jpg', '1054620089.jpg', '105493603.jpg', '1055623002.jpg', '1055753357.jpg', '1057089366.jpg', '106300392.jpg', '106356264.jpg', '106372282.jpg', '1063866640.jpg', '106514190.jpg', '1065323785.jpg', '1066252238.jpg', '106691539.jpg', '1068427675.jpg', '106959209.jpg', '107022557.jpg', '1071201387.jpg', '1072439304.jpg', '1073145694.jpg', '1075716537.jpg', '1075867198.jpg', '1079742005.jpg', '1082250005.jpg', '1082252566.jpg', '1082379191.jpg', '1082

In [15]:
# Now, we need to compute the embeddings
# To speed things up, we destribute pre-computed embeddings
# Otherwise you can also encode the images yourself.
# To encode an image, you can use the following code:
# from PIL import Image
# img_emb = model.encode(Image.open(filepath))

# Here we load the multilingual CLIP model. Note, this model can only encode text.
# If you need embeddings for images, you must load the 'clip-ViT-B-32' model
text_model = SentenceTransformer('clip-ViT-B-32-multilingual-v1')

#For embedding images, we need the non-multilingual CLIP model
img_model = SentenceTransformer('clip-ViT-B-32')

#img_names = list(glob.glob(f'{img_folder}*.jpg'))
test_imgs = list(d_imgs_w_texts.keys())
print("Images:", len(test_imgs))

#img_emb = img_model.encode([Image.open(filepath) for filepath in img_names], batch_size=32, convert_to_tensor=True, show_progress_bar=True) # OSError: [Errno 24] Too many open files
img_embs = []
for i in range(len(test_imgs)):
    print(i,end='\r')
    img_emb = img_model.encode(Image.open(img_folder+test_imgs[i]), convert_to_tensor=True)
    img_embs.append(img_emb)

# tanáré
#encoded_images = np.empty([len(test_imgs), 512])
#for idx, file in enumerate(test_imgs):
#    oimg = Image.open(img_folder+file)
#    print(idx,end='\r')
#    encoded_images[idx] = img_model.encode(oimg)


ftfy or spacy is not installed using BERT BasicTokenizer instead of ftfy.


Images: 6357


In [22]:
# Next, we define a search function.
def search(query, k=5):
    # First, we encode the query (which can either be an image or a text string)
    query_emb = text_model.encode([query], convert_to_tensor=True, show_progress_bar=False)
    
    # Then, we use the util.semantic_search function, which computes the cosine-similarity
    # between the query embedding and all image embeddings.
    # It then returns the top_k highest ranked images, which we output
    hits = util.semantic_search(query_emb, img_embs, top_k=k)[0]
    
    print("Query:")
    display(query)
    for hit in hits:
        print(test_imgs[hit['corpus_id']])
        #display(IPImage((img_folder+test_imgs[hit['corpus_id']]), width=200))


In [23]:
search("két srác együtt ugrott át egy kapun")

Query:


'két srác együtt ugrott át egy kapun'

111061041.jpg
3601394864.jpg
98377566.jpg
293879742.jpg
430623653.jpg
